#**Answer Generation 任務**
資料集引用台達閱讀理解資料集









### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [1]:
!pip install torch torchvision

In [2]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

### **取得 google drive 存取權限**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration/

/content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration


In [5]:
!ls

albert	predict.ipynb  test.json  trained_model  train.ipynb  train.json


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

##下載Albert並放在albert目錄

In [7]:
!git clone https://github.com/harry83528/albert-zh-for-pytorch-transformers.git albert

fatal: destination path 'albert' already exists and is not an empty directory.


In [0]:
def use_model(model_name, config_file_path, model_file_path, vocab_file_path, num_labels):
    # 選擇模型並加載設定
    if(model_name == 'bert'):
        from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
        model_config, model_class, model_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
        tokenizer = model_tokenizer(vocab_file=vocab_file_path)
        return model, tokenizer
    elif(model_name == 'albert'):
        from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForMaskedLM
        model_config, model_class, model_tokenizer = (AlbertConfig, AlbertForMaskedLM, AlbertTokenizer)
        config = model_config.from_pretrained(config_file_path)
        #config = model_config.from_pretrained(config_file_path,num_labels = num_labels) # AlbertForMaskedLM 不需分類,不用設num_labels
        model = model_class.from_pretrained(model_file_path, config=config)
        tokenizer = model_tokenizer.from_pretrained(vocab_file_path)
        return model, tokenizer

### **指定想要用的模型是哪一種**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

In [0]:
#通常我會在這設一些通用模型參數
# PRETRAINED_MODEL_Name ="bert-base-chinese"  # 指定 想要用的預訓練模型
trained_Model_Path = "trained_model" #選擇您儲存的model路徑

## **引入需要⽤到的函式庫**

In [0]:
import os
import pickle
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數
import time

## **預測**

In [0]:
#predict 時把輸入的⽂字轉成輸入格式
def to_input_id(sentence_input):
    #return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input))) # 此任務我們自己組special_token
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input))

In [42]:
#設定模型參數 Start =============================================================
#Step1:初始化加載模型 Start======================================================
model_setting = {
    "model_name":"albert", 
    "config_file_path": trained_Model_Path + "/config.json", 
    "model_file_path": trained_Model_Path + "/pytorch_model.bin", 
    "vocab_file_path":"albert/albert_tiny/vocab.txt",
    "num_labels": -1 # 分幾類(AlbertForMaskedLM 不需分類,不用設num_labels,在這隨便亂設)
}
#初始化加載模型
model, tokenizer = use_model(**model_setting)
#Step1:初始化加載模型 END========================================================

#Step2:指定硬體裝置 Start========================================================
# setting device
#你電腦的 GPU 能否被 PyTorch 調用,如果不行就使用CPU 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device",device)
model.to(device)
#Step2:指定硬體裝置 END==========================================================

model.eval()

#print("輸入您想輸入的context")
context ="古典天體力學研究自然界天體的軌道運動和繞質心運動。19世紀末，研究太陽系中大行星運動和月球運動的理論都已完善，總結出軌道攝動理論。太空飛行器軌道運動理論是在這些理論基礎上發展起來的。由於控制自然天體的基本力量是萬有引力，而人造飛行器自載的動力，因不同於自然的慣性力的作用，故衍生出太空動力學之學門，為軌道力學之重要課題。迄今，人造天體的軌道運動理論仍是天體力學的研究課題。隨著火箭技術的發展，從20世紀1950年代起，人造地球衛星、月球探測器、太空探測器相繼發射成功，軌道運動理論的研究發展成為與工程實踐密切聯繫的應用學科，研究內容也超出了天體力學的傳統範圍。"
#context = input()
#print("輸入您想輸入的question")
#question = input()
question ="什麼力量為控制自然天體的基本？"
input_id = to_input_id("[CLS]" + context + "[SEP]" + question + "[SEP][MASK]")

k = 20 # 取前k個有可能的答案

answer = ""
masked_index = len(input_id)-1 # 要預測的第一個字的索引

# 補齊長度512
while len(input_id)<512:
  input_id.append(0)

#假設答案長度最多15
count = 0
while(count < 15):
    print('input_id:',len(input_id),input_id)
    input_id_tensor=torch.tensor([input_id], dtype=torch.long)
    input_id_tensor = input_id_tensor.to(device) # 資料從CPU RAM 推到 GPU RAM
    outputs = model(input_id_tensor)
    #print('outputs',outputs)

    predictions = outputs[0]
    probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
    indicesList=indices.tolist()
    print('indicesList:',indicesList)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indicesList)
    print('predicted_tokens',type(predicted_tokens),predicted_tokens)

    if predicted_tokens[0] == '[SEP]': #當生成出[SEP]結束
        break

    answer = answer + predicted_tokens[0]
    # 串接預測到的字並再最後的位置補上<MASK>的id 103
    input_id[masked_index] = indicesList[0]
    masked_index = masked_index + 1
    
    if masked_index < 512:
        input_id[masked_index] = 103
    else:
        break

    count += 1
    print('==='*20)

print("生成的答案:" + answer)

using device cuda
input_id: 512 [101, 1367, 1073, 1921, 7768, 1213, 2119, 4777, 4955, 5632, 4197, 4518, 1921, 7768, 4638, 6724, 6887, 6880, 1240, 1469, 5254, 6549, 2552, 6880, 1240, 511, 8131, 686, 5145, 3314, 8024, 4777, 4955, 1922, 7382, 5143, 704, 1920, 6121, 3215, 6880, 1240, 1469, 3299, 4413, 6880, 1240, 4638, 4415, 6316, 6963, 2347, 2130, 1587, 8024, 5244, 5178, 1139, 6724, 6887, 3109, 1240, 4415, 6316, 511, 1922, 4958, 7606, 6121, 1690, 6724, 6887, 6880, 1240, 4415, 6316, 3221, 1762, 6857, 763, 4415, 6316, 1825, 4843, 677, 4634, 2245, 6629, 889, 4638, 511, 4507, 3176, 2971, 1169, 5632, 4197, 1921, 7768, 4638, 1825, 3315, 1213, 7030, 3221, 5857, 3300, 2471, 1213, 8024, 5445, 782, 6863, 7606, 6121, 1690, 5632, 6734, 4638, 1240, 1213, 8024, 1728, 679, 1398, 3176, 5632, 4197, 4638, 2715, 2595, 1213, 4638, 868, 4500, 8024, 3125, 6122, 4495, 1139, 1922, 4958, 1240, 1213, 2119, 722, 2119, 7271, 8024, 4158, 6724, 6887, 1213, 2119, 722, 7028, 6206, 6307, 7539, 511, 6812, 791, 8024, 782, 

In [13]:
print('predict end')

predict end
